# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 21 2023 8:30AM,257783,10,0086408800,ISDIN Corporation,Released
1,Feb 21 2023 8:30AM,257783,10,0086408811,ISDIN Corporation,Released
2,Feb 21 2023 8:30AM,257783,10,0086408812,ISDIN Corporation,Released
3,Feb 21 2023 8:30AM,257783,10,0086408817,ISDIN Corporation,Released
4,Feb 21 2023 8:30AM,257783,10,0086408824,ISDIN Corporation,Released
5,Feb 21 2023 8:30AM,257783,10,0086408836,ISDIN Corporation,Released
6,Feb 21 2023 8:30AM,257782,10,0086414585,ISDIN Corporation,Released
7,Feb 21 2023 8:30AM,257782,10,0086414580,ISDIN Corporation,Released
8,Feb 21 2023 8:30AM,257782,10,0086414581,ISDIN Corporation,Released
9,Feb 21 2023 8:30AM,257783,10,0086408796,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,257799,Released,32
32,257800,Released,45
33,257801,Released,1
34,257802,Released,16
35,257803,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257799,NaN,NaN,32.0
257800,NaN,NaN,45.0
257801,NaN,NaN,1.0
257802,NaN,NaN,16.0
257803,NaN,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257705,1.0,9.0,12.0
257711,0.0,1.0,52.0
257713,0.0,0.0,37.0
257715,0.0,0.0,56.0
257719,0.0,0.0,62.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257705,1,9,12
257711,0,1,52
257713,0,0,37
257715,0,0,56
257719,0,0,62


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257705,1,9,12
1,257711,0,1,52
2,257713,0,0,37
3,257715,0,0,56
4,257719,0,0,62


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257705,1,9,12
1,257711,,1,52
2,257713,,,37
3,257715,,,56
4,257719,,,62


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation
6,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation
67,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation
70,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated"
71,Feb 21 2023 8:29AM,257803,10,Emerginnova
77,Feb 21 2023 8:25AM,257802,10,ISDIN Corporation
93,Feb 21 2023 8:20AM,257800,10,ISDIN Corporation
138,Feb 21 2023 8:20AM,257799,10,ISDIN Corporation
170,Feb 21 2023 8:19AM,257798,10,ISDIN Corporation
196,Feb 21 2023 8:19AM,257797,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,,,10
1,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,,,58
2,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,,,2
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",,,1
4,Feb 21 2023 8:29AM,257803,10,Emerginnova,,,6
5,Feb 21 2023 8:25AM,257802,10,ISDIN Corporation,,,16
6,Feb 21 2023 8:20AM,257800,10,ISDIN Corporation,,,45
7,Feb 21 2023 8:20AM,257799,10,ISDIN Corporation,,,32
8,Feb 21 2023 8:19AM,257798,10,ISDIN Corporation,,,26
9,Feb 21 2023 8:19AM,257797,10,ISDIN Corporation,,,33


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10,,
1,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58,,
2,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2,,
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1,,
4,Feb 21 2023 8:29AM,257803,10,Emerginnova,6,,
5,Feb 21 2023 8:25AM,257802,10,ISDIN Corporation,16,,
6,Feb 21 2023 8:20AM,257800,10,ISDIN Corporation,45,,
7,Feb 21 2023 8:20AM,257799,10,ISDIN Corporation,32,,
8,Feb 21 2023 8:19AM,257798,10,ISDIN Corporation,26,,
9,Feb 21 2023 8:19AM,257797,10,ISDIN Corporation,33,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10,,
1,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58,,
2,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2,,
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1,,
4,Feb 21 2023 8:29AM,257803,10,Emerginnova,6,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10.0,NaN,NaN
1,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58.0,NaN,NaN
2,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2.0,NaN,NaN
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,Feb 21 2023 8:29AM,257803,10,Emerginnova,6.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 21 2023 8:30AM,257783,10,ISDIN Corporation,10.0,0.0,0.0
1,Feb 21 2023 8:30AM,257782,10,ISDIN Corporation,58.0,0.0,0.0
2,Feb 21 2023 8:30AM,257784,10,ISDIN Corporation,2.0,0.0,0.0
3,Feb 21 2023 8:30AM,257778,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,Feb 21 2023 8:29AM,257803,10,Emerginnova,6.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6959424,550.0,10.0,1.0
16,515532,1.0,1.0,0.0
19,515595,22.0,1.0,0.0
22,257792,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6959424,550.0,10.0,1.0
1,16,515532,1.0,1.0,0.0
2,19,515595,22.0,1.0,0.0
3,22,257792,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,550.0,10.0,1.0
1,16,1.0,1.0,0.0
2,19,22.0,1.0,0.0
3,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,550.0
1,16,Released,1.0
2,19,Released,22.0
3,22,Released,1.0
4,10,Executing,10.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,22
Status,,,,
Completed,1.0,0.0,0.0,0.0
Executing,10.0,1.0,1.0,0.0
Released,550.0,1.0,22.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,22
0,Completed,1.0,0.0,0.0,0.0
1,Executing,10.0,1.0,1.0,0.0
2,Released,550.0,1.0,22.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,22
0,Completed,1.0,0.0,0.0,0.0
1,Executing,10.0,1.0,1.0,0.0
2,Released,550.0,1.0,22.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()